In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda 사용가능한지
print(f'Using {device} device')

Using cpu device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() # super함수: 자식 클래스에서 부모 클래스 사용
        self.flatten = nn.Flatten() # reshape를 모델 내에서 동작시켜주는 것, 행렬을 배열로 바꿈
        self.linear_relu_stack = nn.Sequential( # 계층을 선형으로 쌓는 모델?
            nn.Linear(28*28, 512),
            nn.ReLU(), # (x<0, 0, x>0, x)
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    # 입력데이터에 대한 연산 구현
    
    def forward(self, x): 
        x = self.flatten(x)
        logits = self.linear_relu_stack(x) # 
        return logits

In [4]:
model = NeuralNetwork().to(device) # 인공신경망의 인스턴스를 device로 이동
print(model) #구조 출력

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # 모델에 입력, 각 class에 대한 원시(raw) 예측값이 있는 10-차원 텐서 반환
pred_probab = nn.Softmax(dim=1)(logits) # sofmax는 분류에 쓰는 활성화 함수, 0부터 1사이의 실수로 출력, 확률로 표현가능
y_pred = pred_probab.argmax(1)
print(f"Predictaed class: {y_pred}")

Predictaed class: tensor([6])


In [6]:
input_image = torch.rand(3,28,28) # 28*28 크기의 이미지 3개
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten() # nn.Flatten 계층 초기화
flat_image = flatten(input_image) # 28X28의 2D 이미지를 784 픽셀 값의 연속된 배열로 변환
print(flat_image.size()) 

torch.Size([3, 784])


In [8]:
# weight와 bias를 이용해서 입력에 선형 변환을 적용

layer1 = nn.Linear(in_features=28*28, out_features=20) # 입력의 차원, 출력의 차원(28*28 의 입력에 20의 출력)
hidden1 = layer1(flat_image) # hidden에 layer1 배열 저장
print(hidden1.size())

torch.Size([3, 20])


In [9]:
# 비선형 활성화 (선형 변형 후 적용, nonlinearity 도입)
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[-0.3267,  0.1014, -0.1288, -0.1254,  0.1242, -0.0505,  0.4409,  0.3579,
          0.4032, -0.3803, -0.3605,  0.1310,  0.0701,  0.1179,  0.1117, -0.3050,
          0.3901,  0.1579,  0.2031, -0.3529],
        [-0.1205, -0.0125, -0.3094, -0.2263,  0.3067,  0.2154,  0.3615,  0.1556,
         -0.0759, -0.3906, -0.3138,  0.0128,  0.0030,  0.1311,  0.0769, -0.2788,
          0.6185, -0.0852, -0.3457, -0.1796],
        [-0.1228, -0.0498, -0.2498,  0.2665,  0.3305,  0.2559,  0.4961,  0.0212,
          0.1273, -0.5265, -0.4330, -0.2544,  0.0333, -0.1726,  0.2747, -0.1550,
          0.4374, -0.0291, -0.2057, -0.2920]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1014, 0.0000, 0.0000, 0.1242, 0.0000, 0.4409, 0.3579, 0.4032,
         0.0000, 0.0000, 0.1310, 0.0701, 0.1179, 0.1117, 0.0000, 0.3901, 0.1579,
         0.2031, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3067, 0.2154, 0.3615, 0.1556, 0.0000,
         0.0000, 0.0000, 0.0128, 0.0030, 0.1311, 0.07

In [10]:
# 정의된 데이터 순서대로 모든 모듈을 통해 전달

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1) # dim은 합이 1이 되는 차원을 나타냄
pred_probab = softmax(logits) # pred_probab은 [0,1] 범위

In [12]:
# 신경망(model) 출력

print("Model structure: ", model, "\n\n")

# 신경망 내 매개변수(가중치, 편향) 크기와 값 출력

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: <built-in method size of Parameter object at 0x000001E191CE99A0> | Values : tensor([[-0.0216, -0.0326, -0.0051,  ...,  0.0219,  0.0136,  0.0158],
        [-0.0187,  0.0263,  0.0102,  ...,  0.0004,  0.0118, -0.0292]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: <built-in method size of Parameter object at 0x000001E191CE9C20> | Values : tensor([-0.0045,  0.0024], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: <built-in method size of Parameter object at 0x000001E191CE9BD0> | Values : tensor([[ 0.0339,  0.0388,  0.0342,  ...,  0.0058, -0.0142, -0.0265],
       